In [1]:
import glob

In [2]:
# Make sure the sales data is in the sales_records folder at the same location as this notebook
file_names = glob.glob('sales_records/*')
# Creating the collection_names file, overwriting if already exists
file = open('collection_names.txt', "w")
# List just for printing after, so you don't need to open the file just for a quick check
collection_names = []
for name in file_names:
    # CHECK: make sure no collection names include a comma.
    # If one does, we will need to write \n to the list instead of a comma.
    if ',' in name:
        raise ValueError("Comma detected in {}".format(name))
    clean_name = name.split('sales_records/')[1].split(' - Historical Sales')[0].strip()
    file.write(clean_name)
    file.write(',')
    collection_names.append(clean_name)
file.close()

In [3]:
collection_names

['Anonymice',
 'Vox Collectibles',
 'Galacticapes',
 'Irenedao',
 'Byoland',
 'Lonely Alien Space Club',
 '888 Inner Circle',
 'Mfer',
 'Farmers World',
 'Duskbreakers',
 'Lobsterdao',
 'Lil Heroes',
 'Jenkins The Valet The Writers Room',
 'Bored Ape Kennel Club',
 'Mutant Ape Yacht Club',
 'Pudgy Penguins',
 'Gft Atari 50Th Anniversary',
 'Stackedtoadz',
 'Ethereum Name Service',
 'Knights Of Degen',
 'Creaturetoadz',
 'The Other Side',
 'Kingfrogs',
 'More Than Gamers',
 'Bastard Gan Punks V2',
 'Janky Heist',
 'Divine Anarchy',
 'Treasure',
 'Foxfam',
 'The Doge Pound',
 'Bricktopians',
 'Generativedungeon',
 '0N1 Force',
 'Nah Fungible Bones',
 'Lostpoets',
 'Prime Ape Planet',
 'The Shiboshis',
 'Creature World Nft',
 'Wolf Game',
 'Braindrops',
 'Chicken Derby',
 'Red Panda Squad',
 'Metahero Universe Generative',
 'Nifty League Degens',
 'Cyberkongz',
 'Deadfellaz',
 'Art Blocks',
 'Moshi Mochi',
 'Alienfrens',
 'The Cryptodads',
 'Metroverse City Block',
 'Rtfkt Mnlth',
 'Chill